In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.layers import CuDNNLSTM, Dense, Input, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import multi_gpu_model

Using TensorFlow backend.


In [2]:
demo = pd.read_csv('dataset/demo.csv')
demo.rename({
    'c0':'gender',
    'c1':'ocp'}, axis=1, inplace=True)

txn = pd.read_csv('dataset/txn.csv')
txn.rename({
    'n3':'date_idx'
}, axis=1, inplace=True)
txn.drop('t0', axis=1, inplace=True)

train_set = pd.read_csv('dataset/train.csv')
test_set = pd.read_csv('dataset/test.csv')

In [3]:
id_ccno = txn[['id', 'old_cc_no']].drop_duplicates()
train_id_ccno = pd.merge(id_ccno, train_set, on='id')

In [4]:
txn_n_daily = txn[['id', 'old_cc_no', 'date_idx', 'n4', 'n5', 'n6', 'n7']].copy()
txn_n_daily['count'] = 1
txn_n_daily = txn_n_daily.groupby(['id', 'old_cc_no', 'date_idx']).sum()

In [5]:
scaled_txn_n_daily = txn_n_daily

In [6]:
scaled_txn_n_daily.head()

n4    n5    n6    n7  count
id old_cc_no date_idx                                
1  7051      1          6000  1236  5997  2700     12
             2         33000   615  3529  1350      6
             3          5000   904  4300  2025      9
             4          9000  1011  5000  2250     10
             5          3000   618  3000  1350      6

In [7]:
txn_scaler = StandardScaler()
scaled_txn_n_daily[['n4', 'n5', 'n6', 'n7', 'count']] = txn_scaler.fit_transform(txn_n_daily)

In [8]:
scaled_txn_n_daily = scaled_txn_n_daily.reset_index().set_index('old_cc_no')

In [40]:
padding_value = txn_scaler.transform([[0, 0, 0, 0, 0]])
def create_daily_seq_from_group(group):
    global i
    i += 1
    if i % 100==0:
        print(i)
    
    seq = np.ones((365, 5)) * padding_value
    group = group.sort_values('date_idx')
    for (_, _, date_idx, n4, n5, n6, n7, count) in group.itertuples():
        seq[int(date_idx) - 1] = [n4, n5, n6, n7, count]
    return seq
def create_daily_seq(txn):
    grouped = txn.groupby(['old_cc_no'])
    seqs = []
    for name, group in grouped:
        seq = create_daily_seq_from_group(group)
        seqs.append(seq)
    seqs = np.array(seqs)
    return seqs

In [41]:
batch_size = 32
def create_data(sample=-1):
    if sample != -1:
        train_old_cc_nos = train_id_ccno['old_cc_no'].sample(sample)
    else:
        train_old_cc_nos = train_id_ccno['old_cc_no']

    train_txn = scaled_txn_n_daily.loc[train_old_cc_nos]
    train_txn = train_txn.sort_values(['old_cc_no', 'date_idx'])

    seqs = create_daily_seq(train_txn)

    labels = train_id_ccno.set_index('old_cc_no').loc[train_old_cc_nos].sort_values('old_cc_no')['label']

    labels = np.array([to_categorical(label, 13) if not np.isnan(label) else to_categorical(0, 13) for label in labels])
    return seqs, labels
def data_gen(batch_size):
    
    while True:
        seqs, labels = create_data(32*250)
        for i in range(0, len(seqs), batch_size):
            yield seqs[i: i+batch_size], labels[i: i+batch_size]
        

In [42]:
#%%
seqs, labels = create_data(-1)

26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
29200
29300
29400
29500
29600
29700
29800
29900
30000
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
34100
34200
34300
34400
34500
34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
36600
36700
36800
36900
37000
37100
37200
37300
37400
37500
37600
37700
37800
37900
38000
38100
38200
38300
38400
38500
38600
38700
38800
38900
39000
39100
39200
39300
39400
39500
39600
39700
39800
39900
40000
40100
40200
40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500
41600
41700
41800
41900
42000
42100
42200
42300
42400
42500
42600
42700
42800
42900
43000
4310

154100
154200
154300
154400
154500
154600
154700
154800
154900
155000
155100
155200
155300
155400
155500
155600
155700
155800
155900
156000
156100
156200
156300
156400
156500
156600
156700
156800
156900
157000
157100
157200
157300
157400
157500
157600
157700
157800
157900
158000
158100
158200
158300
158400
158500
158600
158700
158800
158900
159000
159100
159200
159300
159400
159500
159600
159700
159800
159900
160000
160100
160200
160300
160400
160500
160600
160700
160800
160900
161000
161100
161200
161300
161400
161500
161600
161700
161800
161900
162000
162100
162200
162300
162400
162500
162600
162700
162800
162900
163000
163100
163200
163300
163400
163500
163600
163700
163800
163900
164000
164100
164200
164300
164400
164500
164600
164700
164800
164900
165000
165100
165200
165300
165400
165500
165600
165700
165800
165900
166000
166100
166200
166300
166400
166500
166600
166700
166800
166900
167000
167100
167200
167300
167400
167500
167600
167700
167800
167900
168000
168100
168200
168300

In [45]:
np.save('seq-daily.npy', seqs)
np.save('label-daily.npy', labels)


In [38]:
%%time
create_data(10000)

CPU times: user 15.2 s, sys: 56.1 ms, total: 15.2 s
Wall time: 15.2 s


(array([[[-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         ...,
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ]],
 
        [[-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         ...,
         [-0.09853715, -0.52710085, -0.87574152,  0.53652621,
          -0.37475338],
         [-0.10382726, -1.69268419, -0.87574152, -1.08722314,
          -1.6825234 ],
         [-0.08972031, 

In [39]:
15.2 * 120

1824.0

In [47]:
model = Sequential()
model.add(CuDNNLSTM(32, return_sequences=True, input_shape=(365, 5)))
model.add(CuDNNLSTM(32, return_sequences=True))
model.add(CuDNNLSTM(32, return_sequences=True))
model.add(CuDNNLSTM(32))
model.add(Dense(13, activation='softmax'))
model.summary()
# model = multi_gpu_model(model, gpus=3)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
             metrics=['acc'])


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_5 (CuDNNLSTM)     (None, 365, 32)           4992      
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 365, 32)           8448      
_________________________________________________________________
cu_dnnlstm_7 (CuDNNLSTM)     (None, 365, 32)           8448      
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (None, 32)                8448      
_________________________________________________________________
dense_2 (Dense)              (None, 13)                429       
Total params: 30,765
Trainable params: 30,765
Non-trainable params: 0
_________________________________________________________________


In [ ]:
class_weights = [1.09518711e+01, 3.07683547e+00, 3.58078232e-01, 2.61727260e-01,
       4.38833908e-01, 1.35073077e+03, 6.72563039e+00, 9.89546351e+00,
       3.99230769e+00, 1.03902367e+01, 4.49295078e-01, 4.26546559e+01,
       1.18329459e+00]

model.fit(seqs,
          labels,
          shuffle=True,
          class_weight=class_weights,
          epochs=100)

Epoch 1/100
 39456/150511 [======>.......................] - ETA: 2:09 - loss: 1.8031 - acc: 0.2999

In [37]:
model.fit_generator(data_gen(32), steps_per_epoch=150000//32, epochs=32)

Epoch 1/32
 678/4687 [===>..........................] - ETA: 6:44 - loss: 1.8307 - acc: 0.2841

KeyboardInterrupt: 